# Investigating perplexity scores for promoter regions using varying promoter extraction techniques

In [1]:
import Bio
from Bio import SeqIO, SeqFeature
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## Genbank files and general genome info

In [2]:
# 5GB1c genome (Methylomicrobium buryatense from Mitch)
gbFile_5G = 'data/5GB1c.current.gb'

# E. coli genome (https://www.ncbi.nlm.nih.gov/nuccore/NC_000913.2)
gbFile_ecoli = 'data/ecoli_k12_NC_000913.2.gb'

In [3]:
gb_5G = SeqIO.parse(gbFile_5G, "genbank").__next__()
gb_ecoli = SeqIO.parse(gbFile_ecoli, "genbank").__next__()

print("___ 5G ____")
print("Genome length:", len(gb_5G.seq), "bps")
print("num features:", len(gb_5G.features))
print("num CDS features:", len([x for x in gb_5G.features if x.type=='CDS']))
print("num gene features:", len([x for x in gb_5G.features if x.type=='gene']))

print("\n___ Ecoli ____")
print("Genome length:", len(gb_ecoli.seq), "bps")
print("num features:", len(gb_ecoli.features))
print("num CDS features:", len([x for x in gb_ecoli.features if x.type=='CDS']))
print("num gene features:", len([x for x in gb_ecoli.features if x.type=='gene']))

___ 5G ____
Genome length: 4998879 bps
num features: 8863
num CDS features: 4369
num gene features: 4427

___ Ecoli ____
Genome length: 4639675 bps
num features: 9913
num CDS features: 4321
num gene features: 4497


In [4]:
# a sampling of genbank features
for x in gb_5G.features[:5]:
    print(x)
    
for x in gb_ecoli.features[:5]:
    print(x)

type: source
location: [0:4998879](+)
qualifiers:
    Key: collection_date, Value: ['1999-01']
    Key: country, Value: ['Russia: Transbaikal']
    Key: db_xref, Value: ['taxon:95641']
    Key: isolation_source, Value: ['Soda lake sediment']
    Key: mol_type, Value: ['genomic DNA']
    Key: organism, Value: ['Methylomicrobium buryatense']
    Key: strain, Value: ['5GB1C']

type: gene
location: [0:1317](+)
qualifiers:
    Key: gene, Value: ['dnaA']
    Key: locus_tag, Value: ['EQU24_00005']

type: CDS
location: [0:1317](+)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: gene, Value: ['dnaA']
    Key: inference, Value: ['COORDINATES: similar to AA sequence:RefSeq:WP_006891211.1']
    Key: locus_tag, Value: ['EQU24_00005']
    Key: note, Value: ['Derived by automated computational analysis using gene prediction method: Protein Homology.']
    Key: product, Value: ['chromosomal replication initiator protein DnaA']
    Key: protein_id, Value: ['PRJNA515283:EQU24_00005']
    Key: tr

## CDS processing

In [5]:
def get_cds_coords(seq_record):
    '''
    Given a SeqRecord parsed from a genbank file, return a list of 
    all the CDS start/end coordinates and gene/locus names
    '''
    cds_list = []
    # Loop over the genome CDS features on each of the strands
    for feature in seq_record.features:
        if feature.type == 'CDS':
            # get  locus tag and gene name
            lt = feature.qualifiers['locus_tag'][0]
            g = "" if 'gene' not in feature.qualifiers else feature.qualifiers['gene'][0]
            
            cds_list.append((feature.location.start.position,
                             feature.location.end.position,
                             feature.strand,
                             lt,
                             g))
    return cds_list

In [6]:
cds_5G = get_cds_coords(gb_5G)
print(cds_5G[:10])
print()

cds_ecoli = get_cds_coords(gb_ecoli)
print(cds_ecoli[:10])

[(0, 1317, 1, 'EQU24_00005', 'dnaA'), (1502, 2603, 1, 'EQU24_00010', ''), (3060, 4140, 1, 'EQU24_00015', 'recF'), (4185, 6600, 1, 'EQU24_00020', 'gyrB'), (6825, 7062, 1, 'EQU24_00025', ''), (7098, 7257, 1, 'EQU24_00030', ''), (7350, 7734, 1, 'EQU24_00035', ''), (7818, 9075, 1, 'EQU24_00040', ''), (9071, 10241, 1, 'EQU24_00045', ''), (10240, 13306, 1, 'EQU24_00050', '')]

[(189, 255, 1, 'b0001', 'thrL'), (336, 2799, 1, 'b0002', 'thrA'), (2800, 3733, 1, 'b0003', 'thrB'), (3733, 5020, 1, 'b0004', 'thrC'), (5233, 5530, 1, 'b0005', 'yaaX'), (5682, 6459, -1, 'b0006', 'yaaA'), (6528, 7959, -1, 'b0007', 'yaaJ'), (8237, 9191, 1, 'b0008', 'talB'), (9305, 9893, 1, 'b0009', 'mog'), (9927, 10494, -1, 'b0010', 'yaaH')]
